## Estimating a sound wave

In this notebook, we read in a recording of a single note on a violin, we estimate it using small number of cosine functions, we plot both for comparison, and we also make both available for listening.

In [1]:
import pyaudio
import wave
import numpy as np
import matplotlib.pyplot as plt

from scipy.fft       import dct, idct
from ipywidgets      import interactive, SelectionSlider
from IPython.display import display, Audio

%matplotlib inline

#### Reading the WAV file
Below, we read in an entire example wave file. We convert it into a numpy array of floats for later manipulation.

In [2]:
p = pyaudio.PyAudio()

wf    = wave.open("data/fiddleg2.wav", "r")
chunk = wf.getnframes()

signal  = wf.readframes(chunk)

data_np = np.frombuffer(signal, dtype=np.uint8).astype(float)-128.0
max_amp = max(data_np)
data_np = data_np/max_amp

display(Audio(data_np, rate=wf.getframerate()))

**Clicking above, you can listen to a single note on a violin from the original wav file.**

Directly below, we define a function we'll use for an interactive plot.

In [3]:
def plot_funcs(num):
        max_coef = int(num)
        fig, (ax, ax2) = plt.subplots(2, figsize=(15,13))
        x = np.arange(0, 2*chunk, 2)
        line,  = ax.plot(x, np.random.rand(chunk), '-', lw=1)
        line2, = ax2.plot(x, np.random.rand(chunk), '-', lw=1)
        ax.set_title('Original Waveform')
        ax.set_xlabel('time')
        ax.set_ylabel('volume')
        ax.set_ylim(-1,1)
        ax.set_xlim(0,chunk)
        ax2.set_title('Estimated Waveform')
        ax2.set_xlabel('time')
        ax2.set_ylabel('volume')
        ax2.set_ylim(-1,1)
        ax2.set_xlim(0,chunk)
        cutoff = -ys[max_coef-1]
        for i in range(chunk):
            if abs(y[i]) < cutoff:
                y_trunc[i]=0.0
            else:
                y_trunc[i]=y[i] 
        yr = idct(y_trunc, norm='ortho')
        line2.set_ydata(yr) 
        line.set_ydata(data_np)
        yr = idct(y_trunc, norm='ortho')
        yr = yr / max(abs(yr))
        frames = yr
        fig.canvas.draw()
        fig.canvas.flush_events()
        plt.show(block=True)
        display(Audio(frames, rate=wf.getframerate()))


#### Descrete cosine transform
In the next cell, we perform a descrete cosine transform of the original signal. This represents the original signal as a linear sum of cosine functions. If we use as many cosine functions as sampling points from the original wave file, we don't lose any quality. When we use fewer functions, it progressively simplifies the sound.

\begin{equation*}
f(t) \approx \sum_{k=1}^n a_kcos(b_kt)   
\end{equation*}

In [4]:
y       = dct(data_np, norm='ortho')
ys      = sorted(-abs(y))
y_trunc = np.zeros(chunk,dtype=float)

myoptions = [int(2**x) for x in np.arange(0, np.log2(chunk))] + [chunk]
interactive_plot = interactive(plot_funcs, num=SelectionSlider(options=myoptions, value=1, 
                                                               description='N cosines'))
output = interactive_plot.children[-1]
output.layout.height = '800px'
output.layout.width  = '800px'
interactive_plot

interactive(children=(SelectionSlider(description='N cosines', options=(1, 2, 4, 8, 16, 32, 64, 128, 256, 512,…

**You can use the slider above to change the number of cosine functions used to represent the sound.**